In [ ]:
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import argparse
import random
import pickle
import cv2
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from google.colab import drive

class ConvAutoencoder:
	@staticmethod
	def build(width, height, depth, filters=(32, 64), latentDim=16):
		# initialize the input shape to be "channels last" along with
		# the channels dimension itself
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1

		# define the input to the encoder
		inputs = Input(shape=inputShape)
		x = inputs

		# loop over the number of filters
		for f in filters:
			# apply a CONV => RELU => BN operation
			x = Conv2D(f, (3,3), strides=1, padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)

		# flatten the network and then construct our latent vector
		volumeSize = K.int_shape(x)
		x = Flatten()(x)
		latent = Dense(latentDim)(x)

		# build the encoder model
		encoder = Model(inputs, latent, name="encoder")


		# start building the decoder model which will accept the
		# output of the encoder as its inputs
		latentInputs = Input(shape=(latentDim,))
		x = Dense(np.prod(volumeSize[1:]))(latentInputs)
		print(volumeSize)
		x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

		# loop over our number of filters again, but this time in
		# reverse order
		for f in filters[::-1]:
			print(f)
			# apply a CONV_TRANSPOSE => RELU => BN operation
			x = Conv2DTranspose(f, (3,3), strides=1,
				padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)

		# apply a single CONV_TRANSPOSE layer used to recover the
		# original depth of the image
		x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
		outputs = Activation("sigmoid")(x)

		# build the decoder model
		decoder = Model(latentInputs, outputs, name="decoder")

		# our autoencoder is the encoder + decoder
		autoencoder = Model(inputs, decoder(encoder(inputs)),
			name="autoencoder")

		encoder.summary()
		print("=====================")
		decoder.summary()
		print("==========================")
		autoencoder.summary()
		# return a 3-tuple of the encoder, decoder, and autoencoder
		return (encoder, decoder, autoencoder)
  
# USAGE
# python train_unsupervised_autoencoder.py --dataset output/images.pickle --model output/autoencoder.model

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")



def batch_generator(Train_dt,batch_size,steps):
	idx = 1
	while True:
		yield getImages(batch_size, Train_dt, idx-1)
		if idx<steps:
			idx += 1
		else:
			idx = 1
    	
    	
    	
def getImages(batchSize, folderName,idx):
	names = os.listdir(folderName)
	skiprows = int(idx*batchSize)
	batchout = []
	start = skiprows
	end = skiprows +batchSize
	while start<end :
		batchout.append(names[start])
		start += 1
	images = []
	for i in batchout:
		images.append(np.array(Image.open(os.path.join(folderName,i))))
	return (np.array(images), np.array(images))
 
"""
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", type=str,default="lfwautoencoder_latest.model",
	help="path to output trained autoencoder")
ap.add_argument("-p", "--plot", type=str, default="plot_latest.png",
	help="path to output plot file")
args = vars(ap.parse_args())
"""

# initialize the number of epochs to train for, initial learning rate,
# and batch size

# construct our convolutional autoencoder
INIT_LR = 1e-6
print("[INFO] building autoencoder...")
BS = 32
nb_epochs = 10
train_data = os.listdir("/content/drive/My Drive/data/train")
steps_per_epoch_train = (len(train_data)//BS)

my_training_batch_generator = batch_generator("/content/drive/My Drive/data/train", BS,steps_per_epoch_train)
val_data = os.listdir("/content/drive/My Drive/data/val")
steps_per_epoch_val = len(val_data)//BS
my_val_batch_generator = batch_generator("/content/drive/My Drive/data/val",BS,steps_per_epoch_val)

(encoder, decoder, autoencoder) = ConvAutoencoder.build(250, 250, 3)
opt = Adam(lr=INIT_LR, decay=INIT_LR / nb_epochs)
autoencoder.compile(loss="mse", optimizer=opt)

# train the convolutional autoencoder
H = autoencoder.fit_generator(
	my_training_batch_generator,
	epochs=nb_epochs,steps_per_epoch = steps_per_epoch_train,verbose=1,
	validation_data= my_val_batch_generator,validation_steps = steps_per_epoch_val)
# use the convolutional autoencoder to make predictions on the
# testing images, construct the visualization, and then save it
# to disk


# construct a plot that plots and saves the training history
N = np.arange(0, nb_epochs)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig('my_figure.png')

# serialize the image data to disk
print("[INFO] saving image data...")

autoencoder.save_weights('./checkpoints/my_checkpoint')


[INFO] building autoencoder...
(None, 250, 250, 64)
64
32
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 250, 250, 32)      896       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 250, 250, 32)      0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 250, 250, 32)      128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 250, 250, 64)      18496     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 250, 250, 64)      0         
_________________________________________________________________
b